In [1]:
# Importing the pdfplumber module for extracting tables from PDFs.
import pdfplumber

# Importing the re module for regular expressions.
import re

# Importing the csv module for working with CSV files.
import csv

In [2]:
# Path to the PDF file
pdf_path = '/Users/joathcarrera/Desktop/CSE115A/Soccer-Match-Predictor/2425_Calendars/Bundesliga_Calendar.pdf'
# Path to the output CSV file
base_csv_path = '/Users/joathcarrera/Desktop/CSE115A/Soccer-Match-Predictor/2425_Calendars/Matchday_Calendars/bundesliga_2024-2025fixtures.csv'

In [3]:
def extract_tables_from_pdf(pdf_path):
    """
    This function takes a path to a PDF as input and returns a list of
    tables extracted from the PDF.

    The function opens the PDF using the pdfplumber library and iterates
    over each page in the PDF. For each page, it extracts all the tables
    and adds them to the list of tables.

    The list of tables is then returned.
    """
    tables = []
    with pdfplumber.open(pdf_path) as pdf:
        # Iterate over each page in the PDF
        for page_num, page in enumerate(pdf.pages):
            # Extract all the tables from the current page
            page_tables = page.extract_tables()
            # Add each extracted table to the list of tables
            for table in page_tables:
                tables.append(table)
    # Return the list of tables
    return tables

In [4]:
def clean_date(text):
    """
    This function takes a text string representing a date and returns a formatted date string.
    If the input text does not match any of the recognized date patterns, the original text is returned.
    """

    # Define the date patterns to match against the input text
    # These patterns are used to extract the date information from the input text
    date_range_pattern = r'(\d{2})\.(\d{2})\.-(\d{2})\.(\d{2})\.(\d{4})'  # Matches dates in the format dd.mm.-dd.mm.yyyy
    date_range_slash_pattern = r'(\d{2})\./(\d{2})\.(\d{2})\.(\d{4})'  # Matches dates in the format dd/mm/dd.yyyy
    date_range_hyphen_pattern = r'(\d{2})\.-(\d{2})\.(\d{2})\.(\d{4})'  # Matches dates in the format dd-mm-dd.yyyy
    single_date_pattern = r'(\d{2})\.(\d{2})\.(\d{4})'  # Matches dates in the format dd.mm.yyyy

    # Check if the input text matches any of the date patterns
    if (match := re.match(date_range_pattern, text)):  # Check if the input text matches the date_range_pattern
        # If it matches, extract the start and end dates
        start_day, start_month, end_day, end_month, year = match.groups()
        # Format the start and end dates
        start_date = f"{start_month}/{start_day}/{year}"  # Format the start date
        end_date = f"{end_month}/{end_day}/{year}"  # Format the end date
        # Return the formatted date range
        return f"{start_date} - {end_date}"

    elif (match := re.match(date_range_slash_pattern, text)):  # Check if the input text matches the date_range_slash_pattern
        # If it matches, extract the start and end dates
        start_day, end_day, month, year = match.groups()
        # Format the start and end dates
        start_date = f"{month}/{start_day}/{year}"  # Format the start date
        end_date = f"{month}/{end_day}/{year}"  # Format the end date
        # Return the formatted date range
        return f"{start_date} - {end_date}"

    elif (match := re.match(date_range_hyphen_pattern, text)):  # Check if the input text matches the date_range_hyphen_pattern
        # If it matches, extract the start and end dates
        start_day, end_day, month, year = match.groups()
        # Format the start and end dates
        start_date = f"{month}/{start_day}/{year}"  # Format the start date
        end_date = f"{month}/{end_day}/{year}"  # Format the end date
        # Return the formatted date range
        return f"{start_date} - {end_date}"

    elif (match := re.match(single_date_pattern, text)):  # Check if the input text matches the single_date_pattern
        # If it matches, extract the date
        day, month, year = match.groups()
        # Format the date
        date = f"{month}/{day}/{year}"  # Format the date
        # Return the formatted date
        return date

    else:  # If the input text does not match any of the date patterns
        # Return the original input text
        return text

In [5]:
def clean_and_split_tables(tables):
    """
    This function takes a list of tables as input and returns a dictionary of tables grouped by matchday.
    It cleans the date information in each row and checks if the third element represents a matchday.
    If it does, it adds the row to the appropriate matchday list.
    """
    # Create a dictionary to store the tables grouped by matchday
    tables_by_matchday = {}

    # Iterate over each table in the input list
    for table in tables:
        # Iterate over each row in the current table
        for row in table:
            # Check if the row has at least two elements
            if row and len(row) > 1:
                # Clean the date information in the row
                cleaned_row = [clean_date(cell) if cell else cell for cell in row]

                # Check if the third element of the row represents a matchday
                if cleaned_row[2] and re.match(r"\d+", cleaned_row[2]):
                    # Pad the matchday with zeros to ensure it has two digits
                    matchday = cleaned_row[2].zfill(2)

                    # If the matchday is not already in the dictionary, create an empty list for it
                    if matchday not in tables_by_matchday:
                        tables_by_matchday[matchday] = []

                    # Append the cleaned row to the list of rows for the current matchday
                    tables_by_matchday[matchday].append(cleaned_row)

    # Return the dictionary of tables grouped by matchday
    return tables_by_matchday

In [6]:
def filter_bundesliga_matches(matchday_tables):
    """
    This function takes a dictionary of tables grouped by matchday as input
    and returns a new dictionary with the tables filtered to only include
    Bundesliga matches.

    The function iterates over each table in the input dictionary and
    checks if the fifth element of each row (index 4) contains any
    of the keywords in the exclude_keywords set. If the row has more
    than three elements and the fifth element is not empty, the row
    is added to the filtered_table list.

    If the filtered_table list is not empty, it is added to the
    filtered_tables dictionary with the current matchday as the
    key.

    The function returns the filtered_tables dictionary.
    """
    
    # Create an empty dictionary to store the filtered tables
    filtered_tables = {}
    
    # Define the set of keywords to exclude from the matches
    exclude_keywords = {"DFB", "DFL", "UECL", "A", "UCL", "UEL", "REL"}
    
    # Iterate over each table in the input dictionary
    for matchday, table in matchday_tables.items():
        
        # Create an empty list to store the filtered rows for the current matchday
        filtered_table = []
        
        # Iterate over each row in the current table
        for row in table:
            
            # Check if the row has more than three elements and the fifth element is not empty
            if len(row) > 3 and row[4]:
                
                # Convert the fifth element to lowercase and replace any whitespace with a single space
                lower_case_row = re.sub(r'\s+', ' ', row[4]).lower()
                
                # Check if any of the exclude_keywords are in the lowercase row
                if not any(keyword in lower_case_row for keyword in exclude_keywords):
                    
                    # If no keywords are found, add the row to the filtered_table list
                    filtered_table.append(row)
        
        # If the filtered_table list is not empty, add it to the filtered_tables dictionary
        if filtered_table:
            filtered_tables[matchday] = filtered_table
    
    # Return the filtered_tables dictionary
    return filtered_tables

In [7]:
def save_matchday_tables_to_csv(matchday_tables, base_csv_path):
    """
    Save each matchday table to a separate CSV file.

    This function takes a dictionary of tables grouped by matchday as input
    and a base path for the CSV files. It iterates over each table in the
    input dictionary, generates a CSV file path for each table using the
    matchday and the base path, and writes the table data to the CSV file.

    Args:
        matchday_tables (dict): Matchday tables grouped by matchday.
        base_csv_path (str): Base path for the CSV files.
    """
    # Iterate over each matchday and table in the matchday_tables dictionary
    for matchday, table in sorted(matchday_tables.items()):
        # Generate a CSV file path for the current matchday and base path
        csv_path = f"{base_csv_path}_matchday_{matchday}.csv"

        # Open the CSV file in write mode with UTF-8 encoding
        with open(csv_path, mode='w', newline='', encoding='utf-8') as file:
            # Create a CSV writer object
            writer = csv.writer(file)

            # Write the header row to the CSV file
            writer.writerow(["Date", "Matchday", "Home Team", "Away Team"])

            # Iterate over each row in the current table
            for row in table:
                # Check if the row has at least five elements and is not the header row
                if len(row) >= 5 and row != ["Datum", "Spieltag", "Heim", "Gast"]:
                    # Write the row data to the CSV file
                    writer.writerow([row[0], row[2], row[4], row[5]])

In [8]:
# Extract tables from the PDF
tables = extract_tables_from_pdf(pdf_path)

In [9]:
# Clean and split the tables into matchday tables
matchday_tables = clean_and_split_tables(tables)

In [10]:
# Filter the matchday tables to include only Bundesliga matches
filtered_matchday_tables = filter_bundesliga_matches(matchday_tables)

In [11]:
# Save the filtered matchday tables to CSV files
save_matchday_tables_to_csv(filtered_matchday_tables, base_csv_path)

In [12]:
# Print a success message indicating that the filtered matchday tables have been saved to CSV files
print("Filtered Bundesliga matchday tables have been saved to CSV files.")

Filtered Bundesliga matchday tables have been saved to CSV files.
